In [ ]:
from jupyter_bbox_widget import BBoxWidget
import ipywidgets as widgets
import os
import json

# Introduction

Initialize a widget with an image path and a list of classes.

Click and drag anywhere on the image to create bboxes, move and resize them as necessary.

In [ ]:
widget = BBoxWidget(
    image='fruit.jpg',
    classes=['apple', 'orange', 'pear'],
)
widget

Access the boxes you created like this:

In [ ]:
widget.bboxes

# A simple workflow example

Let's say we have a folder of image files that we would like to create annotations for.

In [ ]:
path = 'relative/path/to/my/images'
files = sorted(os.listdir(path))

annotations = {}
annotations_path = 'annotations.json'

We'll use `BBoxWidget` for creating annotations for an image. Let's also add a progress bar and "Save" and "Skip" buttons for going through our list of images. For this we'll use widgets from `ipywidgets` library.

In [ ]:
# a progress bar to show how far we got
w_progress = widgets.IntProgress(value=0, max=len(files), description='Progress')
# a save button
w_save = widgets.Button(description="Save", button_style='success', icon='check')
# a skip button
w_skip = widgets.Button(description="Skip", icon='arrow-right')
# the bbox widget
w_bbox = BBoxWidget(
    image = os.path.join(path, files[0]),
    classes=['cat', 'dog', 'whatever']
)

# combine all the widgets into a container
w_container = widgets.VBox([
    widgets.HBox([
        w_progress,
        w_skip,
        w_save,
    ]),
    w_bbox
])

Define the functions to process clicks on our Save and Skip buttons.

In [ ]:
# when Skip button is pressed we move on to the next file
def on_skip(_):
    w_progress.value += 1
    # open new image in the widget
    image_file = files[w_progress.value]
    w_bbox.image = os.path.join(path, image_file)
    # here we assign an empty list to bboxes but 
    # we could also run a detection model on the file
    # and use its output for creating inital bboxes
    w_bbox.bboxes = [] 
w_skip.on_click(on_skip)

# when Save button is pressed we save current annotations
# and then move on to the next file
def on_save(_):
    image_file = files[w_progress.value]
    # save annotations for current image
    annotations[image_file] = w_bbox.bboxes
    with open('annotations.json', 'w') as f:
        json.dump(annotations, f, indent=4)
    # move on to the next file
    on_skip(0)
w_save.on_click(on_save)

Now we display the container widget and we are ready to annotate!

In [ ]:
w_container